In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from data.dataset import ModelNet40, collate_fn
import os
import numpy as np

from utils import *
from model2 import *
from torch.utils.data import DataLoader, Subset
import torch
from evaluate import *
from train import train

In [3]:
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'
device

'cuda'

### Getting the dataset path

In [4]:
os.chdir('..')
modelnet40_path = os.path.join(os.getcwd(), 'ModelNet40')
os.chdir('TransformersFor3dPointCLouds')
print(modelnet40_path)

/scratch/users/ahamadeh18/COMP390/ModelNet40


### Initalizing the training set DataLoader object

In [5]:
SAMPLE_SIZE = 128
TEST = False
SAMPLING_METHOD = 'fps'
batch_size = 5

In [6]:
dataset = ModelNet40(dataset_path=modelnet40_path, test=TEST, sample_size=SAMPLE_SIZE, sampling=SAMPLING_METHOD)

In [7]:
class_ind_dist = dataset.class_indicies_distribution()

In [8]:
overfit_classes_num = 4
OVERFITTING_SAMPLE_SIZE = 10
overfitting_set = []
for i in range(overfit_classes_num):
    overfitting_set += class_ind_dist[i][:OVERFITTING_SAMPLE_SIZE]

In [9]:
overfitting_subset = Subset(dataset, overfitting_set)
overfitting_loader = DataLoader(overfitting_subset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

## Overfitting the model

In [10]:
from torch.nn import CrossEntropyLoss
from torch.nn import NLLLoss

class_weights =  torch.tensor(dataset.class_weights).float()
criterion = CrossEntropyLoss(weight=class_weights)
# criterion = CrossEntropyLoss()

In [11]:
in_features = 3
feature_dim = 128
out_features = 1024
decoder_features = 256
k_size = 4
NUM_CLASSES = 40

In [12]:
model = PointCloudClassifier(in_features, feature_dim, out_features, decoder_features, k_size, NUM_CLASSES)

self.in_features: 3 | feature_dim: 128


In [13]:
from torch.optim import SGD
from torch.optim.lr_scheduler import CosineAnnealingLR

parameters = model.parameters()
learning_rate = 1e-2
momentum = 0.9
weight_decay = 1e-7

optimizer = SGD(parameters, lr=learning_rate, momentum=momentum)

step = len(dataset)
scheduler = CosineAnnealingLR(optimizer, step)

In [14]:
epochs = 20
loss = train(model=model, optimizer=optimizer,
          scheduler=scheduler, train_loader=overfitting_loader,
          criterion=criterion, epochs=epochs, verbose=False)

100%|██████████| 8/8 [00:18<00:00,  2.34s/it]


epoch: 0 | loss: 29.414411544799805
Epoch time: 18.74873161315918


100%|██████████| 8/8 [00:12<00:00,  1.57s/it]


epoch: 1 | loss: 31.444308280944824
Epoch time: 12.59646463394165


100%|██████████| 8/8 [00:12<00:00,  1.62s/it]


...

epoch: 17 | loss: 30.78080177307129
Epoch time: 10.545141220092773


100%|██████████| 8/8 [00:14<00:00,  1.75s/it]


epoch: 18 | loss: 29.70242929458618
Epoch time: 14.020538806915283


100%|██████████| 8/8 [00:10<00:00,  1.32s/it]

epoch: 19 | loss: 29.693613529205322
Epoch time: 10.58909273147583
Final loss 29.693613529205322


### Plotting the loss:

In [ ]:
import matplotlib.pyplot as plt
plt.plot(loss)

In [ ]:
accuracy = eval_2(model, overfitting_loader)

In [ ]:
print(f'{accuracy}%')

In [ ]:
x = dataset[0][0].view(1, 128, 3)
yhat = model(x.float())
yhat

In [ ]:
softmax = torch.nn.Softmax()(yhat)
softmax

In [ ]:
torch.argmax(softmax)

### Creating the model

In [21]:
in_features = 3
feature_dim = 128
out_features = 1024
decoder_features = 256
k_size = 4
NUM_CLASSES = 40

In [22]:
model = PointCloudClassifier(in_features, feature_dim, out_features, decoder_features, k_size, NUM_CLASSES)

self.in_features: 3 | feature_dim: 128


### Dataset

In [23]:
SAMPLE_SIZE = 128
TEST = False
SAMPLING_METHOD = 'fps'

model_net40 = ModelNet40(dataset_path=modelnet40_path, test=TEST, sample_size=SAMPLE_SIZE, sampling=SAMPLING_METHOD)

### Optimizer:

In [24]:
from torch.optim import SGD
from torch.optim.lr_scheduler import CosineAnnealingLR

parameters = model.parameters()
learning_rate = 1e-3
momentum = 0.9

optimizer = SGD(parameters, lr=learning_rate, momentum=momentum)

step = len(model_net40)
scheduler = CosineAnnealingLR(optimizer, step)

## Training the model

### DataLoader

In [25]:
batch_size = 128

In [26]:
from torch.utils.data import DataLoader

train_loader = DataLoader(model_net40, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

### Optimizer:

In [27]:
from torch.optim import SGD
from torch.optim.lr_scheduler import CosineAnnealingLR

parameters = model.parameters()
learning_rate = 1e-4
momentum = 0.9

optimizer = SGD(parameters, lr=learning_rate, momentum=momentum)

step = len(model_net40)
scheduler = CosineAnnealingLR(optimizer, step)

### Loss function:

In [28]:
from torch.nn import CrossEntropyLoss
from torch.nn import NLLLoss

class_weights = torch.tensor(model_net40.class_weights).float()
criterion = CrossEntropyLoss(weight=class_weights)

In [ ]:
epochs = 10
loss = train(model=model, optimizer=optimizer,
          scheduler=scheduler, train_loader=train_loader,
          criterion=criterion, epochs=epochs, save_params=True, verbose=False, load_model=False)

100%|██████████| 77/77 [33:08<00:00, 25.82s/it]


epoch: 0 | loss: 290.4781391620636
Epoch time: 1988.5203080177307


 27%|██▋       | 21/77 [07:39<18:52, 20.22s/it]

## Model Evaluation:

### Loading the test set:

In [ ]:
SAMPLE_SIZE = 128
TEST = True
SAMPLING_METHOD = 'fps'
batch_size = 1

model_net_256_test = ModelNet40(dataset_path=modelnet40_path, test=TEST, sample_size=SAMPLE_SIZE, sampling=SAMPLING_METHOD)
test_laoder = DataLoader(model_net_256_test, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

In [ ]:
evaluate(model, model_net_256_test, epochs, verbose=True)

In [ ]:
data, label, _ = model_net_256_test[0]

In [ ]:
yhat = model(data.float())
print(label)
print((torch.argmax(yhat) == label).item())

In [ ]:
accuracy = 0
N = len(model_net_256_test)
model.eval()
for point in model_net_256_test:
    if point == None:
        N -= 1
        continue
    x, y, _ = point
    yhat = model(x.float())
    
    if (torch.argmax(yhat) == y).item():
        accuracy += 1

In [ ]:
acc = accuracy / N

In [ ]:
acc